# Jira Client Tutorial

In [15]:
import os
import sys
from pathlib import Path

# since notebook is outside of the src directory, we need to add the src directory to the path
project_root = Path.home() / "dev" / "data-analyser"
if project_root not in sys.path:
    sys.path.append(str(project_root))

from dotenv import load_dotenv

from src.clients.jira_client import JiraClient

load_dotenv()

True

In [16]:
# JIRA connection parameters
JIRA_BASE_URL = os.environ.get("JIRA_BASE_URL")
JIRA_USER_EMAIL = os.environ.get("JIRA_USER_EMAIL")
JIRA_API_TOKEN = os.environ.get("JIRA_API_TOKEN")
JIRA_PROJECT_KEY = os.environ.get("JIRA_PROJECT_KEY")

In [17]:
# Jira Client setup
jira = JiraClient(
    base_url=JIRA_BASE_URL, email=JIRA_USER_EMAIL, api_token=JIRA_API_TOKEN
)

In [19]:
# teesting
import requests
from requests.auth import HTTPBasicAuth
import os

def delete_issues_for_project(project_key: str, base_url: str, email: str, api_token: str) -> None:
    """
    Deletes all issues for the specified project in Jira.
    
    Args:
        project_key (str): The key of the Jira project (e.g., "DATA").
        base_url (str): The base URL of the Jira instance (e.g., "https://yourcompany.atlassian.net").
        email (str): Your Jira user email.
        api_token (str): Your Jira API token.
    
    Raises:
        Exception: If any HTTP request fails.
    """
    auth = HTTPBasicAuth(email, api_token)
    headers = {"Accept": "application/json", "Content-Type": "application/json"}
    jql = f"project={project_key}"

    start_at = 0
    max_results = 50

    while True:
        # Search issues
        search_url = f"{base_url}/rest/api/3/search"
        params = {"jql": jql, "startAt": start_at, "maxResults": max_results}
        response = requests.get(search_url, headers=headers, auth=auth, params=params)

        if response.status_code != 200:
            raise Exception(f"Failed to search issues: {response.text}")

        data = response.json()
        issues = data.get("issues", [])

        if not issues:
            print("No more issues to delete.")
            break

        for issue in issues:
            issue_key = issue["key"]
            delete_url = f"{base_url}/rest/api/3/issue/{issue_key}"
            del_response = requests.delete(delete_url, headers=headers, auth=auth)

            if del_response.status_code == 204:
                print(f"Deleted issue: {issue_key}")
            else:
                print(f"Failed to delete issue {issue_key}: {del_response.text}")

        start_at += max_results


In [22]:
delete_issues_for_project(
    project_key="KAN",
    base_url=JIRA_BASE_URL,
    email=JIRA_USER_EMAIL,
    api_token=JIRA_API_TOKEN
)


Deleted issue: KAN-14
Deleted issue: KAN-13
Deleted issue: KAN-12
Deleted issue: KAN-11
Deleted issue: KAN-10
Deleted issue: KAN-9
No more issues to delete.


In [ ]:
JIRA_BASE_URL
JIRA_USER_EMAIL
JIRA_API_TOKEN
JIRA_PROJECT_KEY

In [4]:
# Issue creation
issue_info = {
    "project": JIRA_PROJECT_KEY,
    "summary": "Car Models Analysis",
    "description": "How many unqiue car models we have per car category? Sort the results in descending order!",
    "issuetype": "Task",
}

resposne = jira.create_issue(issue_info)
resposne

{'id': '10018',
 'key': 'KAN-8',
 'self': 'https://vladislavphysique.atlassian.net/rest/api/2/issue/10018'}

In [5]:
issue = jira.get_issue(issue="KAN-9")
issue_details = jira.extract_issue_details(issue)

In [ ]:
# change ticket status
jira.transition_issue(issue_key=issue_details["ticket_id"], status_name="В работе")

# add comment
jira.add_comment(issue_key=issue_details["ticket_id"], comment="Starting the analysis!")

In [8]:
jira.add_comment(issue_key=issue_details["ticket_id"], comment="Starting the analysis!")

{'ticket_id': 'KAN-9',
 'summary': 'Car Models Analysis',
 'description': 'How many unqiue car models we have per car category? Sort the results in descending order!',
 'status': 'К выполнению',
 'assignee': {'account_id': None, 'name': None, 'email': None},
 'reporter': {'account_id': '70121:f86554c4-819a-470f-9606-bf3e8b1202e6',
  'name': 'Vladislav Raskoshinskii',
  'email': 'vladislavphysique@gmail.com'},
 'priority': 'Medium',
 'due_date': None,
 'created_date': '2025-06-18T17:47:39.964+0200',
 'updated_date': '2025-06-18T17:47:40.039+0200'}

In [6]:
jira.update_issue(issue_id=issue_details.ticket_id, status="In Progress")

{'ticket_id': 'KAN-9',
 'summary': 'Car Models Analysis',
 'description': 'How many unqiue car models we have per car category? Sort the results in descending order!',
 'status': 'К выполнению',
 'assignee': {'account_id': None, 'name': None, 'email': None},
 'reporter': {'account_id': '70121:f86554c4-819a-470f-9606-bf3e8b1202e6',
  'name': 'Vladislav Raskoshinskii',
  'email': 'vladislavphysique@gmail.com'},
 'priority': 'Medium',
 'due_date': None,
 'created_date': '2025-06-18T17:47:39.964+0200',
 'updated_date': '2025-06-18T17:47:40.039+0200'}